For now on, I'll need a faster way to manage the database. This test will check accesibility for datasets with HDF5


In [2]:
!pip install simpleITK

     |████████████████████████████████| 47.4MB 86kB/s 


In [30]:
# Git Configuration
!git config --global user.email 'obandomarcos96@gmail.com'
!git config --global user.password 'Quovadis96'

In [31]:
!git add Utilities/DataLoading.py Tests/MoDL_training_test.ipynb Tests/HDF5_dataset_test.ipynb; git commit -m '#9 option 1, saving datasets with built-in Pandas method'

[main 415f299] #9 option 1, saving datasets with built-in Pandas method
 3 files changed, 45 insertions(+), 8 deletions(-)
 create mode 100644 Tests/HDF5_dataset_test.ipynb
 rewrite Tests/MoDL_training_test.ipynb (97%)


In [1]:
import os
os.chdir('/content/drive/MyDrive/Balseiro/Maestría/Implementación/DeepOPT')
# import some libraries
import os,time, sys
sys.path.append('Utilities/')

import numpy as np
import h5py as h5
%load_ext autoreload
%autoreload 1
import matplotlib.pyplot as plt
import pandas as pd
%aimport DataLoading
DL = DataLoading

In [2]:
f140114_5dpf = "/content/drive/MyDrive/Balseiro/Maestría/Implementación/Datos/140114_5dpf"  # 5 days post-fertilization
f140117_3dpf = "/content/drive/MyDrive/Balseiro/Maestría/Implementación/Datos/140117_3dpf"  # 3 days post-fertilization
f140115_1dpf = "/content/drive/MyDrive/Balseiro/Maestría/Implementación/Datos/140315_1dpf"  # 1 days post-fertilization

f140315_3dpf = "/content/drive/MyDrive/Balseiro/Maestría/Implementación/Datos/140315_3dpf"     # 3 days post-fertilization
f140415_5dpf_4X = "/content/drive/MyDrive/Balseiro/Maestría/Implementación/Datos/140415_5dpf_4X"  # 5 days post-fertilization
f140419_5dpf = "/content/drive/MyDrive/Balseiro/Maestría/Implementación/Datos/140519_5dpf"     # 5 days post-fertilization

f140714_5dpf = "/content/drive/MyDrive/Balseiro/Maestría/Implementación/Datos/140714_5dpf"
f140827_3dpf_4X = "/content/drive/MyDrive/Balseiro/Maestría/Implementación/Datos/140827_3dpf_4X"
f140827_5dpf_4X = '/content/drive/MyDrive/Balseiro/Maestría/Implementación/Datos/140827_5dpf_4X'

folder_paths = [f140114_5dpf]
results_folder = '/content/drive/MyDrive/Balseiro/Maestría/Implementación/1-DL/Resultados/'

In [87]:
dfs = [DL.ZebraDataset(dataset, 'Datasets', 'Bassi') for dataset in folder_paths]

for df in dfs:
  
  # Cargo el dataset
  df.loadImages(sample = 'head')
  # Cargo las registraciones correspondientes
  df.loadRegTransforms()
  # Aplico las transformaciones para este dataset 
  df.applyRegistration(sample = 'head')

100%|██████████| 180/180 [00:14<00:00, 12.21it/s]


In [88]:
volume = df.getRegisteredVolume('head')

In [93]:
with h5.File('Datasets/OPTdatasets.hdf5', 'r') as f:
  print(f.keys())

<KeysViewHDF5 ['Bassi']>


In [89]:
def getDataset(experimentName, ):
  '''
  Gets

On branch main
Your branch is ahead of 'origin/main' by 1 commit.
  (use "git push" to publish your local commits)

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)
  (commit or discard the untracked or modified content in submodules)

	modified:   Tests/HDF5_dataset_test.ipynb
	modified:   Utilities/DataLoading.py
	modified:   modl (modified content, untracked content)

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	Datasets/
	DatasetsOPTdatasets.hdf5
	Utilities/__pycache__/

no changes added to commit (use "git add" and/or "git commit -a")


In [ ]:
!git add Tests/HDF5_dataset_test.ipynb Utilities/DataLoading.py; git commit -m '#9 hdf5'

In [56]:
with h5.File(df.datasetFolder+'/'+'OPTdatasets.hdf5', 'r') as datasets_file:

  print(datasets_file['Bassi']['140114_5dpf']['head']['block1_values'])

[128   4 149 ... 148  98  46]


In [58]:
with h5.File(df.datasetFolder+'/'+'OPTdatasets.hdf5', 'r') as datasets_file:
  print(datasets_file['Bassi']['140114_5dpf']['head'].keys())

<KeysViewHDF5 ['axis0', 'axis1', 'block0_items', 'block0_values', 'block1_items', 'block1_values']>
